**DSCI100 Final Project Report**

Group 19 (Aneeta, Avishka, Hannah, Ksenia)

**Introduction**

In this project, analysis on data from the High Time Resolution Universe Pulsar Survey  will be done to gain a better understanding of how to better classify Neutron stars (also known as pulsars).

Studying pulsars allows researchers to explore “space-time, the interstellar medium, and states of matter.” However, radio signals received to identify pulsars are obstructed with radio frequency interference (RFI) and noise. As a result, there is demand for advanced learning tools that will be able to automatically label pulsar candidates.

If the pulse’s signal is integrated with respect to its rotational period, a unique integrated profile can be created for the pulse. A DM-SNR curve plots the spectral supernova remnants (SNR) as a function of the trail dispersion measure (DM).

The question that this project will answer is “Can an unclassified candidate be classified as a pulsar based on observed characteristics?" The characteristics that will be used in this classification, consisting of information from both the integrated profile and the DM-SNR curve, will be determined based on which one—integrated profile or DM-SNR curve provides a model with the highest prediction accuracy.

The dataset that is used for this project provides information on 17,898 candidates—16,259 are examples of RFI/noise, and 1,639 are examples of real pulsars. 8 variables corresponding to data about the integrated profile and DM-SNR curve of the candidates, will be further analyzed in the project to answer the research question.


**Methods & Results**

In our exploratory data analysis, we did cross validation while tuning the classifiers for different combinations of classifiers to determine which classifiers at which k value would give us the most accurate model. We then used these parameters to build a model that would classify a candidate for our planned data analysis.


***Loading, cleaning and wrangling the data into R***

1. Load the libraries tidyverse, tidymodels, repr, ggplot2 and cowplot.


In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(ggplot2)
library(cowplot)

2. Read the data from the web using the read_csv() function.
    - Add column names to the dataframe using the mutate() function
    - Change the class to a categorical variable using the as_factor() function.
    - Use the sample_n() function to randomly sample 500 observations from the dataframe.
        - Here we reduce the dataset to a workable size, as there were too many observations initially.
    - Name the output as pulsar_data.

Note: Here we chose not to balance for pulsar and non-pulsar class, as when we conducted the analysis with a variety of different balancing methods, this resulted in unusual elbow-plots in a later stage. This unbalanced method also produced high quality confusion matrices in a later step.

***Splitting our data into training and testing sets***

3. Set seed as 19 for each consecutive cell.
4. Split the data into two sets in a ratio of 3:1 using the initial_split() function. Name the output as pulsar_split.
5. Create training and testing data sets by passing the object pulsar_split into the training and testing functions. Name the outputs as pulsar_training_data and pulsar_testing_data respectively.


***Creating a knn classification model specification***

6. Create a knn classification model specification with a straight line as weight function and tuning on the number of neighbors. Name the output as knn_spec.


***Exploratory data analysis***

Here we are looking to see which value using which combination of predictors gives us the highest prediction accuracy, so we can use those parameters to build the most accurate model.

7. Carry out a 5-fold cross-validation on the training data using the vfold_cv() function with “class” in the strata argument and name the output as pulsar_vfold
8. Create a dataframe for k values from 1 to 10 using the tibble() function and name the output as k_values.
9. Create a recipe for the model using the mean and standard deviation of the integrated profile. Name the output as pulsar_ip_recipe_1.
10. Create a workflow analysis with the recipe and model specification using the tune_grid() function. Specify that the tuning should try k_values.
    - Collect the metrics from the workflow analysis using the collect_metrics() function.
    - Filter for accuracy and name the output as knn_ip_results_1.
11. Plot the neighbors k vs the accuracy using the ggplot function. Assign this plot to an object called ip_plot_1.
12. Repeat steps 9 to 11 for each variable pair namely:
    - Mean and Skewness of the Integrated Profile
    - Mean and Excess kurtosis of the Integrated Profile
    - Mean and Standard deviation of the DM-SNR curve
    - Mean and Skewness of the DM-SNR curve
    - Mean and Excess kurtosis of the DM-SNR curve
    - Name the outputs accordingly.


13. Analyze the accuracy vs k plots for each set of variables.
14. To choose the set with the highest accuracy, create a data frame containing the highest accuracy with the corresponding k value for each plot as shown below.
Eg. Mean vs Standard deviation of the Integrated Profile
    - Arrange the mean in knn_ip_results_1 in descending order using the arrange() function.
    - Use the slice() and select() functions to select the highest mean and its corresponding k value for the Mean vs Standard deviation of the Integrated Profile.
    - Add a new column called “variables” and the label "Mean IP & Std dev" to this dataframe using the mutate() function.
    - Name the output as max_ip_dev.
15. Repeat Step 14 for each variable pair. Name the outputs accordingly.
16. Combine the objects max_ip_dev, max_ip_skewness, max_ip_kurtosis, max_dm_dev, max_dm_skewness, and  max_dm_kurtosis in a single data frame using the bind_rows() function.
17. Arrange the data in this new data frame by descending order of mean using the arrange() function. Name the output as max_accuracies.
18. Choosing the first row in max_accuracies shows the best variable pair and its corresponding best k value, which is  the Mean and Excess kurtosis of the Integrated Profile with k = 3.


***Planned data analysis***

We will build our model using Mean IP and Excess kurtosis as predictors, as we determined above

19. Create a specification for our final knn classification model with a straight line as weight function and using the k = 3 in the neighbor argument. Name the output as pulsar_model_spec. 
20. Create a workflow analysis using the model specification pulsar_model_spec  and recipe pulsar_ip_recipe_3 and pass the training data set to the fit() function. Assign the answer to an object called pulsar_model_fit.
21. Use pulsar_model_fitl to predict on the test dataset and assign this to an object called pulsar_predictions.
22. Report the accuracy of this prediction using the metrics() function, and store this in an object named pulsar_metrics.
23. Create a confusion matrix using the conf_mat() function and store this in an object named pulsar_conf_mat.



**Discussions**

As a result of graphing to determine best accuracy, it is determined that mean and excess kurtosis of integrated profile has the highest average, with a k-value of 3. In correspondence to the hypothesis, integrated profile data will be a better predictor than DM-SNR to classify candidates as pulsar or not. Since these parameters gave the highest accuracy 


 It is important to note that the accuracy values are higher than what is typically expected from such an analysis. We believe this is the case as we made the conscious choice to leave our data unbalanced in terms of class, as we were encountering abnormal elbow plots when we were balancing our data with various methods.


As we predicted, integrated profile was an overall more accurate predictor of class than DM-SNR curve. We did not make any predictions about which specific variables would be more reliable. 
It is visually clear that our random point would be assigned as a pulsar star, due to its location on the graph, and the neighboring points around it.
These findings regarding the most reliable predictors, as well as our prediction of a random set of values can be used to make quick and simple predictions regarding data points that fall clearly within the pulsar or non-pulsar areas of the graph. By only using two variables to predict, predictions can be made very quickly.
This finding can also be further expanded on by exploring if including a third predictor would produce more accurate results, especially in the range of our data where both pulsars and non-pulsars currently fall, which may currently be inaccurately predicted.


***Predicting the class of a new observation using our model***

24. Generate random values for the mean and excess kurtosis of an integrated profile using the run_if() function and the minimum and maximum values from the min_max_mean_data.
25. Create a dataframe named random_point using the tibble() function.
26. Predict the class of random_point from the prediction model pulsar_model_fit using the predict() function. Name the output class_prediction.

The class of the random point we created was a pulsar star. All three of its nearest neighbors were classed as pulsars. 


**References**

R. J. Lyon, B. W. Stappers, S. Cooper, J. M. Brooke, J. D. Knowles, Fifty Years of Pulsar Candidate Selection: From simple filters to a new principled real-time classification approach, Monthly Notices of the Royal Astronomical Society 459 (1), 1104-1123, DOI: 10.1093/mnras/stw656	
https://archive.ics.uci.edu/ml/datasets/HTRU2 

M. J. Keith et al., 'The High Time Resolution Universe Pulsar Survey - I. System Configuration and Initial Discoveries',2010, Monthly Notices of the Royal Astronomical Society, vol. 409, pp. 619-627. DOI: 10.1111/j.1365-2966.2010.17325.x

D. R. Lorimer and M. Kramer, 'Handbook of Pulsar Astronomy', Cambridge University Press, 2005.

